In [1]:
# Importing libraries and data sets
!pip install streamlit -q
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
import streamlit as st
import nltk
from sklearn.metrics.pairwise import cosine_similarity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.1 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
book_df = pd.read_csv('/content/drive/MyDrive/Book_Recommendation_System_2/books_with_emotions.csv')
book_df.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348286,0.942528,0.704421,0.887940,0.111690,0.252544
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765


In [3]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5197 entries, 0 to 5196
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   isbn13              5197 non-null   int64  
 1   isbn10              5197 non-null   object 
 2   title               5197 non-null   object 
 3   authors             5165 non-null   object 
 4   categories          5167 non-null   object 
 5   thumbnail           5031 non-null   object 
 6   description         5197 non-null   object 
 7   published_year      5197 non-null   float64
 8   average_rating      5197 non-null   float64
 9   num_pages           5197 non-null   float64
 10  ratings_count       5197 non-null   float64
 11  title_and_subtitle  5197 non-null   object 
 12  tagged_description  5197 non-null   object 
 13  simple_categories   5197 non-null   object 
 14  anger               5197 non-null   float64
 15  disgust             5197 non-null   float64
 16  fear  

In [4]:
book_df = book_df[~book_df['authors'].isna()]

In [5]:
book_df['thumbnail'] = book_df['thumbnail'].fillna('https://nftcalendar.io/storage/uploads/2022/02/21/image-not-found_0221202211372462137974b6c1a.png')
book_df['categories'] = book_df['categories'].fillna('Not Found')

In [6]:
missing_cat = book_df[book_df['categories'].isna()].index.to_list()

In [7]:
for x in missing_cat:
    book_df.loc[x,'categories'] = book_df.loc[x,'simple_categories']

In [8]:
book_df.loc[473,'categories'] = book_df.loc[473,'simple_categories']

In [9]:
book_df.columns.to_list()

['isbn13',
 'isbn10',
 'title',
 'authors',
 'categories',
 'thumbnail',
 'description',
 'published_year',
 'average_rating',
 'num_pages',
 'ratings_count',
 'title_and_subtitle',
 'tagged_description',
 'simple_categories',
 'anger',
 'disgust',
 'fear',
 'joy',
 'neutral',
 'sadness',
 'surprise']

In [10]:
book_df = book_df[book_df['ratings_count']>=5]

In [11]:
book_df = book_df[['isbn13','title','authors','categories','thumbnail','description','average_rating','simple_categories','anger','disgust','fear','joy','neutral','sadness','surprise']]

In [12]:
book_df.head()

,isbn13,title,authors,categories,thumbnail,description,average_rating,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,3.85,Fiction,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,0.729602
1,9780002261982,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,3.83,Fiction,0.612619,0.348286,0.942528,0.704421,0.887940,0.111690,0.252544
2,9780006178736,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",3.93,Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,4.15,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690,0.078765
4,9780006280934,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",4.09,Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765


In [13]:
book_df['authors'] = book_df['authors'].str.replace(" ","")
book_df['authors'] = book_df['authors'].str.replace(";"," ")
book_df['authors'] = book_df['authors'].str.replace("'","")
book_df['authors'] = book_df['authors'].str.replace("-","")
book_df['authors'] = book_df['authors'].str.replace(":","")
book_df['authors'] = book_df['authors'].str.replace(",","")
book_df['authors'] = book_df['authors'].str.replace("’","")
book_df['authors'] = book_df['authors'].str.replace("(","")
book_df['authors'] = book_df['authors'].str.replace(")","")
book_df['authors'] = book_df['authors'].str.lower()

In [14]:
book_df['description'] = book_df['description'].str.replace(";"," ")
book_df['description'] = book_df['description'].str.replace("'","")
book_df['description'] = book_df['description'].str.replace("-","")
book_df['description'] = book_df['description'].str.replace(":","")
book_df['description'] = book_df['description'].str.replace(",","")
book_df['description'] = book_df['description'].str.replace("’","")
book_df['description'] = book_df['description'].str.replace("(","")
book_df['description'] = book_df['description'].str.replace(")","")
book_df['description'] = book_df['description'].str.lower()

In [15]:
book_df['categories'] = book_df['categories'].str.replace(";"," ")
book_df['categories'] = book_df['categories'].str.replace("'","")
book_df['categories'] = book_df['categories'].str.replace("-","")
book_df['categories'] = book_df['categories'].str.replace(":","")
book_df['categories'] = book_df['categories'].str.replace(",","")
book_df['categories'] = book_df['categories'].str.replace("’","")
book_df['categories'] = book_df['categories'].str.replace("(","")
book_df['categories'] = book_df['categories'].str.replace(")","")
book_df['categories'] = book_df['categories'].str.lower()

In [16]:
book_df['tags'] = book_df['authors'] + ' ' + book_df['description'] + ' ' + book_df['average_rating'].astype(str) + ' ' + book_df['categories']

In [17]:
# book_df.to_csv("books.csv", index = False)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english'
)

tfidf_matrix = tfidf.fit_transform(book_df['tags'])


In [19]:
def recommend_books_by_description(user_description, df, tfidf, tfidf_matrix):

    # Preprocess input
    user_description = user_description.lower()

    # Vectorize input
    user_vec = tfidf.transform([user_description])

    # Cosine similarity
    similarity_scores = cosine_similarity(user_vec, tfidf_matrix)[0]

    # Get top N indices
    top_indices = similarity_scores.argsort()[::-1]

    # Return recommended books
    recommendations = df.iloc[top_indices][
        ['isbn13', 'title','thumbnail', 'authors', 'average_rating','simple_categories','anger','disgust','fear', 'joy', 'neutral', 'sadness', 'surprise']]

    return recommendations


In [20]:
user_input = "romance"
top_n = 5
results = recommend_books_by_description(
    user_input, book_df, tfidf, tfidf_matrix)

results = results[:20].reset_index()

In [21]:
for i in range(len(results)):
    print(results['title'][i])
    print(results['thumbnail'][i])

Over the Moon
http://books.google.com/books/content?id=DspEXFnOn1IC&printsec=frontcover&img=1&zoom=1&source=gbs_api
Hard Candy
http://books.google.com/books/content?id=vMVlAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api
The Magyar Venus
http://books.google.com/books/content?id=dpSTvymwLNoC&printsec=frontcover&img=1&zoom=1&source=gbs_api
Sylvester
http://books.google.com/books/content?id=-wO7fVdHF2gC&printsec=frontcover&img=1&zoom=1&source=gbs_api
The History of the Siege of Lisbon
http://books.google.com/books/content?id=5WXe9lE-Ix8C&printsec=frontcover&img=1&zoom=1&source=gbs_api
Blessings
http://books.google.com/books/content?id=XSGjKUwqLSgC&printsec=frontcover&img=1&zoom=1&source=gbs_api
Ivanhoe
http://books.google.com/books/content?id=IiOgdjKRM5oC&printsec=frontcover&img=1&zoom=1&source=gbs_api
And the Envelope, Please...
http://books.google.com/books/content?id=0B2X-s87SFAC&printsec=frontcover&img=1&zoom=1&source=gbs_api
Deep and Meaningful Diaries from Planet Janet
http:

In [22]:
category = "Nonfiction"
if category == " ":
    filter_df = results
else:
    filter_df = results[results['simple_categories'].str.lower() == category.lower()]
filter_df

,index,isbn13,title,thumbnail,authors,average_rating,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
16,1170,9780312049997,The Sun in the Morning,https://nftcalendar.io/storage/uploads/2022/02...,marymargaretkaye,4.21,Nonfiction,0.002809,0.002762,0.004051,0.814095,0.131116,0.017412,0.027756


In [23]:
mood = ["joy","fear"]
filter_df['mood_avg'] = filter_df[mood].mean(axis=1)
filter_df = filter_df.sort_values(ascending=False,by='mood_avg')
filter_df['title'].head()

/tmp/ipython-input-143764997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df['mood_avg'] = filter_df[mood].mean(axis=1)


,title
16,The Sun in the Morning


In [24]:
pkl.dump(book_df,open('Books.pkl','wb'))
pkl.dump(tfidf_matrix,open('similarity_score.pkl','wb'))
pkl.dump(tfidf,open('model.pkl','wb'))

In [25]:
!pip install streamlit -q
!wget -q -O - ipv4.icanhazip.com

34.28.238.205


In [27]:
! streamlit run /content/drive/MyDrive/Book_Recommendation_System_2/app.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.28.238.205:8501

y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://eight-schools-fall.loca.lt
────────────────────────── Traceback (most recent call last) ───────────────────────────
  /usr/local/lib/python3.12/dist-packages/streamlit/runtime/scriptrunner/exec_code.py:  
  129 in exec_func_with_error_handling                                                  
                                                                                        
  /usr/local/lib/python3.12/dist-packages/streamlit/runtime/scriptrunner/script_runner  
  .py:689 in code_to_exec                                                               
                                                                                        
  /content/drive/MyDrive/Book_Recomme